# Assignment #1: PFL067 Statistical NLP

## Exploring Entropy and Language Modeling

---

### 1. Entropy of a Text

In this experiment, you will determine the conditional entropy of the word distribution in a text given the previous word. To do this, you will first have to compute P(i,j), which is the probability that at any position in the text you will find the word i followed immediately by the word j, and P(j|i), which is the probability that if word i occurs in the text then word j will follow. Given these probabilities, the conditional entropy of the word distribution in a text given the previous word can then be computed as:

$$H(J|I) = -\sum_{i \in I, j \in J} P(i,j) \log_2 P(j|i)$$

The perplexity is then computed simply as

$$P_X(P(J|I)) = 2^{H(J|I)}$$

Compute this conditional entropy and perplexity for `TEXTEN1.txt`

This file has every word on a separate line. (Punctuation is considered a word, as in many other cases.) The i,j above will also span sentence boundaries, where i is the last word of one sentence and j is the first word of the following sentence (but obviously, there will be a fullstop at the end of most sentences).

---

In [372]:
import pandas as pd
import numpy as np
import collections as c
import math
import random

In [373]:
english = './TEXTEN1.txt'
czech = './TEXTCZ1.txt'

In [374]:
def open_text(filename):
    """Reads a text line by line, applies light preprocessing, and returns a dataframe of each word"""
    with open(filename) as f:
        content = f.readlines()

    df = pd.DataFrame(content, columns=['words'])
    df['words'] = df['words'].apply(lambda word: word.strip().lower())
    
    return df

In [375]:
list(open_text(english)['words'])[:10]

['when', 'on', 'board', 'h', '.', 'm', '.', 's', '.', 'beagle']

In [376]:
def language_model(words):
    """Counts unigrams and bigrams in a dataframe"""
    word_counts = c.Counter(words)
    num_words = sum(word_counts.values())
    vocabulary = list(set(word_counts.keys()))

    bigrams = list(zip(['<s>'] + words, words + ['</s>']))
    bigram_counts = c.Counter(bigrams)
    num_bigrams = sum(bigram_counts.values())
    bigram_vocabulary = list(set(bigram_counts.keys()))
    
    unigram_model = words, word_counts, num_words, vocabulary
    bigram_model = bigrams, bigram_counts, num_bigrams, bigram_vocabulary
    
    return (unigram_model, bigram_model)

In [377]:
def Pword(unigram_model, W='', alpha=0.7):
    """Calculates the probability a word appears in a sentence"""
    _, word_counts, num_words, vocabulary = unigram_model
    return (word_counts[W] + alpha) / (num_words + alpha * len(vocabulary))

In [378]:
def Pbigram(bigram_model, W='', Wprev='', alpha=0.7):
    """Calculates the probability a bigram appears in a sentence"""
    _, bigram_counts, num_bigrams, bigram_vocabulary = bigram_model
    return (bigram_counts[(Wprev, W)] + alpha) / (num_bigrams + alpha * len(bigram_vocabulary))

In [379]:
# P(A|B) = P(A,B) / P(B)
def Pwprev(models, W='', Wprev=''):
    """Calculates the probability a word W proceeds a word Wprev"""
    unigram_model, bigram_model = models
    return Pbigram(bigram_model, W=W, Wprev=Wprev) / Pword(unigram_model, W=Wprev)

In [380]:
def entropy(models, bigrams):
    """Calculates the entropy from a list of bigrams"""
    _, bigram_model = models
    return - sum(Pbigram(bigram_model, W=W, Wprev=Wprev) 
                 * math.log(Pwprev(models, W=W, Wprev=Wprev), 2) for Wprev,W in bigrams)

In [381]:
def perplexity(models, bigrams):
    """Calculates the perplexity from a list of bigrams"""
    return 2 ** entropy(models, bigrams)

In [388]:
def english_entropy():
    text = open_text(english)
    words = list(text['words'])
    models = language_model(words)
    _, bigram_model = models
    bigrams, *_ = bigram_model

    return entropy(models, bigrams), perplexity(models, bigrams)

In [389]:
english_entropy()

(314.09587888960993, 3.566818908057433e+94)

Next, you will mess up the text and measure how this alters the conditional entropy. For every character in the text, mess it up with a likelihood of 10%. If a character is chosen to be messed up, map it into a randomly chosen character from the set of characters that appear in the text. Since there is some randomness to the outcome of the experiment, run the experiment 10 times, each time measuring the conditional entropy of the resulting text, and give the min, max, and average entropy from these experiments. Be sure to use srand to reset the random number generator seed each time you run it. Also, be sure each time you are messing up the original text, and not a previously messed up text. Do the same experiment for mess up likelihoods of 5%, 1%, .1%, .01%, and .001%.

In [356]:
def charset(text):
    words = text['words']
    return sorted(list(set(char for word in words for char in word)))

In [357]:
charset(open_text(english))[:10]

['!', '"', '&', "'", '(', ')', ',', '.', '/', '0']

In [369]:
def perturb_char(word, charset, prob=0.1, seed=200):
    """Changes each character with given probability to a random character in the charset"""
    np.random.seed(seed)
    random.seed(seed)
    
    arr = list(word)
    for i,char in enumerate(arr):
        if np.random.random() < prob:
            arr[i] = np.random.choice(charset)
            
    return ''.join(arr)

In [370]:
def perturb(text, charset, prob=0.1, seed=200):
    key = str(prob)
    text[key] = text['words'].apply(lambda word: perturb_char(word, charset, prob, seed))
    return text

In [392]:
def perturb_text(text):
    for prob in [0.1, 0.05, 0.01, 0.001, 0.0001, 0.00001]:
        perturb(text, charset(text), prob=prob)
        
    return text

In [393]:
perturb_text(open_text(english))[:10]

,words,0.1,0.05,0.01,0.001,0.0001,1e-05
0,when,when,when,when,when,when,when
1,on,on,on,on,on,on,on
2,board,board,board,board,board,board,board
3,h,h,h,h,h,h,h
4,.,.,.,.,.,.,.
5,m,m,m,m,m,m,m
6,.,.,.,.,.,.,.
7,s,s,s,s,s,s,s
8,.,.,.,.,.,.,.
9,beagle,beagld,beagld,beagld,beagle,beagle,beagle


In [400]:
def english_entropy_perturbed():
    text = open_text(english)
    models = language_model(list(text['words']))
    _, bigram_model = models
    bigrams, *_ = bigram_model
    
    perturb_text(text)
    
    arr = []
    ## TODO: isolate bigrams from LM
    for words in text:
        arr.append((entropy(models, bigrams), perplexity(models, bigrams)))

    return arr

In [401]:
english_entropy_perturbed()

[(314.09587888960993, 3.566818908057433e+94),
 (314.09587888960993, 3.566818908057433e+94),
 (314.09587888960993, 3.566818908057433e+94),
 (314.09587888960993, 3.566818908057433e+94),
 (314.09587888960993, 3.566818908057433e+94),
 (314.09587888960993, 3.566818908057433e+94),
 (314.09587888960993, 3.566818908057433e+94)]

### Questions
1. Should we split the data into train/test sets?
2. What entropy values should we be seeing so that I know I am on the right track?